<a href="https://colab.research.google.com/github/USIS015220/coolab/blob/main/Te_damos_la_bienvenida_a_Colaboratory.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
leguajes = pd.Series([
                      "C#",
                      "VB",
                      "PHP",
                      "HTML",
                      "JS",
                      "CSS",
                      "JAVA",
                      "XML",
                      "Python"],
                     index=[2,1,4,5,6,9,3,8,10])
print(leguajes, leguajes[2])

diccionarioLenguajes = {
    2:"C#",
    1:"VB",
    4:"PHP",
    5:"HTML",
    6:"JS",
    9:"CSS",
    3:"JAVA",
    8:"XML",
    10:"Python"
}
pdLenguajes = pd.Series(diccionarioLenguajes)
print(pdLenguajes)

for i in range(30,2000):
  pdLenguajes[i] = "Lenguaje Favorito "+ str(i)
print(pdLenguajes)

2         C#
1         VB
4        PHP
5       HTML
6         JS
9        CSS
3       JAVA
8        XML
10    Python
dtype: object C#
2         C#
1         VB
4        PHP
5       HTML
6         JS
9        CSS
3       JAVA
8        XML
10    Python
dtype: object
2                           C#
1                           VB
4                          PHP
5                         HTML
6                           JS
                 ...          
1995    Lenguaje Favorito 1995
1996    Lenguaje Favorito 1996
1997    Lenguaje Favorito 1997
1998    Lenguaje Favorito 1998
1999    Lenguaje Favorito 1999
Length: 1979, dtype: object


In [2]:
import pandas as pd
import matplotlib.pyplot as plt

archivo = pd.read_csv("/content/notas.csv", sep=";", encoding="utf-8")
#print(archivo, archivo["notas"])

proceso = pd.crosstab(index=archivo["notas"],columns="frecuencia")
#print(proceso)

sobresalientes = proceso.loc[proceso.index>=6]["frecuencia"]
print(sobresalientes)
plt.pie(sobresalientes, labels=sobresalientes.index)
plt.xlabel("Estudiantes que pasaron")
plt.show()

grupoNotas = archivo.groupby(["notas"]).agg({"sum"})
print(grupoNotas)
grupoNotas.to_csv("/content/grupoNotas.csv", sep=";")

#plt.pie(proceso, labels=proceso.index)
#plt.xlabel("Estudiantes sobresalientes")
#plt.show()

FileNotFoundError: ignored

In [3]:
import pandas as pd
import matplotlib.pyplot as plt

usuarios = pd.read_csv("/content/usuarios.csv", sep=";")
#print(usuarios["pais"])

peliculas = pd.read_csv("/content/peliculas.csv", sep=";")
#print(peliculas)

usuario_pelicula = pd.read_csv("/content/usuario_peliculas.csv", sep=";")
#print(usuario_pelicula)

usuarioPelicula =pd.merge( pd.merge(usuarios,usuario_pelicula), peliculas, on="idPelicula")
print(usuarioPelicula)

FileNotFoundError: ignored